## Sentiment Analysis

### Business Problem

- This analysis will aim to build a model that can rate the sentiment of a Tweet based on its content.
### Objectives

- To build a multimodal classifier that will accurately classify tweets into positive, negative and neutral

## Importing Relevant Libraries


In [31]:
import pandas as pd

## Loading Data

### Importing the Dataset

In [32]:
# Set display options to show all rows and increase the column width
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Read data
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding='latin1')

In [33]:
# Checking the first five rows
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [34]:
# Simplifying the column names

df.columns = ['Tweet','Brand','Emotion']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    9092 non-null   object
 1   Brand    3291 non-null   object
 2   Emotion  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


The dataset contains a total of 9,093 tweets, with nearly all entries having text data. However, only 3,291 entries specify a brand or product, which highlights that many tweets do not directly mention a particular brand. Despite this, each tweet is associated with an emotion, either positive or negative, which helps to understand the sentiment being conveyed.

## Data Cleaning

### Cleaning of the Brand column

In [35]:
# Identify distribution of column
df['Brand'].value_counts()

Brand
iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: count, dtype: int64

In [36]:
# Map product category to brand
product_mapping = {
    "iPad": "Apple",
    "iPad or iPhone App": "Apple",
    "iPhone": "Apple",
    "Other Apple product or service": "Apple",
    "Other Google product or service": "Google",
    "Android App": "Google",
    "Android": "Google"
}

# Map the 'Brand' column to 'Brand' using the product mapping
df["Brand"] = df["Brand"].map(product_mapping).fillna(df["Brand"])

# Display value counts and info
print(df["Brand"].value_counts())
print(df.info())


Brand
Apple     2409
Google     882
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    9092 non-null   object
 1   Brand    3291 non-null   object
 2   Emotion  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB
None
